In [17]:
import secret_keys as key
from tqdm import tqdm_notebook as tqdm
import pandas as pd


In [2]:
import tweepy

# Creating the authentication object
auth = tweepy.OAuthHandler(key.consumer_key, key.consumer_secret)
# Setting your access token and secret
auth.set_access_token(key.access_token, key.access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth,wait_on_rate_limit=True) 

In [52]:
search_words = '#corona OR #covid-19 OR #covid OR #flattenthecurve'
date_since = '2020-03-21'

In [53]:
tweets = tweepy.Cursor(api.search ,
                       q = search_words,
                       lang = 'en',
                       since = date_since
                      ).items(1000)

In [54]:
tweet_list = [[ tweet.geo, tweet.text, tweet.user.screen_name, tweet.user.location ] for tweet in tweets]

In [55]:
df_tweets = pd.DataFrame(tweet_list, columns = ['geo', 'tweet' , 'user_handle','location'])

In [56]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
geo            1 non-null object
tweet          1000 non-null object
user_handle    1000 non-null object
location       1000 non-null object
dtypes: object(4)
memory usage: 31.3+ KB


In [58]:
# https://towardsdatascience.com/another-twitter-sentiment-analysis-bb5b01ebad90
# Tweet data cleaning
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [65]:
df_tweets['cleaned_tweets'] = df_tweets.tweet.progress_apply(tweet_cleaner)

In [66]:
df_tweets.head()

geo                                              tweet      user_handle  \
0  None  RT @dustin_SFA: My wife and I pastor a church ...  mmmmaybachmusik   
1  None  RT @UNOCHA: 🚨 MEDIA ALERT\n\n@antonioguterres ...        UNinSyria   
2  None  RT @francesleach_: Cuban doctors arriving into...          Dorli28   
3  None  COVID-19: Union Bank Activates Remote Working ...    tcpmultimedia   
4  None  via @PerilofAfrica  Russian Fitness Model Draw...      mlnangalama   

                location                                     cleaned_tweets  
0                         rt sfa my wife and i pastor a church in sherid...  
1                  Syria  rt media alert will virtually launch the covid...  
2                         rt cuban doctors arriving into italy to suppor...  
3                         covid union bank activates remote working for ...  
4  New Brunswick, Canada  via russian fitness model draws ire after maki...

In [72]:
df_tweets['cleaned_tweets'] = df_tweets.cleaned_tweets.str.replace('rt', '')

In [73]:
df_tweets.head()

geo                                              tweet      user_handle  \
0  None  RT @dustin_SFA: My wife and I pastor a church ...  mmmmaybachmusik   
1  None  RT @UNOCHA: 🚨 MEDIA ALERT\n\n@antonioguterres ...        UNinSyria   
2  None  RT @francesleach_: Cuban doctors arriving into...          Dorli28   
3  None  COVID-19: Union Bank Activates Remote Working ...    tcpmultimedia   
4  None  via @PerilofAfrica  Russian Fitness Model Draw...      mlnangalama   

                location                                     cleaned_tweets  
0                          sfa my wife and i pastor a church in sheridan...  
1                  Syria   media ale will viually launch the covid globa...  
2                          cuban doctors arriving into italy to suppo th...  
3                         covid union bank activates remote working for ...  
4  New Brunswick, Canada  via russian fitness model draws ire after maki...

In [74]:
df_tweets.to_csv('../Data/Tweets_scraped_23rd_CORONA_COVID_FTC.csv')